<a href="https://colab.research.google.com/github/gabpiazza/Big-Bang/blob/main/Analysis_full_panel_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
## 1.1 Install & Load packages --------------------------------------------------------

# some setup: a cheeky little bit of code to check and install packages
need <- c("tidyverse","did", "progress") # list packages needed
have <- need %in% rownames(installed.packages()) # checks packages you have
if(any(!have)) install.packages(need[!have]) # install missing packages
invisible(lapply(need, library, character.only=T)) # load needed packages

options(scipen = 999)

In [9]:
full_panel<- readRDS("/content/full_panel")
names(full_panel
)


[1] "year"                                    
  [2] "bvd_id_number"                           
  [3] "country"                                 
  [4] "filing_type"                             
  [5] "fixed_assets"                            
  [6] "intangible_fixed_assets"                 
  [7] "tangible_fixed_assets"                   
  [8] "current_assets"                          
  [9] "total_assets"                            
 [10] "number_of_employees"                     
 [11] "operating_revenue_turnover_"             
 [12] "p_l_before_tax"                          
 [13] "p_l_after_tax"                           
 [14] "research_development_expenses"           
 [15] "ebitda"                                  
 [16] "year_orbis"                              
 [17] "month_orbis"                             
 [18] "matching_year"                           
 [19] "order_date"                              
 [20] "registration_year"                       
 [21] "total_chf_amount_year"                   
 [22] "max_tech"                                
 [23] "first_order"                             
 [24] "last_order"                              
 [25] "total_orders"                            
 [26] "total_orders_amount"                     
 [27] "first_order_amount"                      
 [28] "first_order_tech"                        
 [29] "registration_first_order"                
 [30] "code_2_digits"                           
 [31] "subproject_first_year"                   
 [32] "subproject_1_first_year"                 
 [33] "consolidation_l"                         
 [34] "first_year"                              
 [35] "last_year"                               
 [36] "order_after_last_orbis"                  
 [37] "year_after_order"                        
 [38] "year_before_order"                       
 [39] "company_name_latin_alphabet"             
 [40] "nace_rev_2_core_code_4_digits"           
 [41] "postcode_latin_alphabet"                 
 [42] "nace_rev_2_main_section"                 
 [43] "size_classification"                     
 [44] "status"                                  
 [45] "branch_indicator"                        
 [46] "longitude"                               
 [47] "latitude"                                
 [48] "status_updated_date"                     
 [49] "national_legal_form"                     
 [50] "last_avail_year"                         
 [51] "incorporation_year"                      
 [52] "status_simple"                           
 [53] "year_patent"                             
 [54] "number_applications"                     
 [55] "number_WIPO_code_apps"                   
 [56] "number_weighted_patent_apps"             
 [57] "number_multiple_inventors_apps"          
 [58] "number_collaborations_apps"              
 [59] "number_multiple_patent_offices_apps"     
 [60] "number_publications"                     
 [61] "number_WIPO_code_pubs"                   
 [62] "number_weighted_patent_pubs"             
 [63] "number_multiple_inventors_pubs"          
 [64] "number_collaborations_pubs"              
 [65] "number_multiple_patent_offices_pubs"     
 [66] "application_stock"                       
 [67] "publication_stock"                       
 [68] "probability_applications"                
 [69] "probability_WIPO_code_apps"              
 [70] "probability_weighted_patent_apps"        
 [71] "probability_multiple_inventors_apps"     
 [72] "probability_collaborations_apps"         
 [73] "probability_multiple_patent_offices_apps"
 [74] "probability_publications"                
 [75] "probability_WIPO_code_pubs"              
 [76] "probability_weighted_patent_pubs"        
 [77] "probability_multiple_inventors_pubs"     
 [78] "probability_collaborations_pubs"         
 [79] "probability_multiple_patent_offices_pubs"
 [80] "log_applications"                        
 [81] "asinh_applications"                      
 [82] "log_WIPO_code_apps"       

In [2]:
`%notin%` <- Negate(`%in%`)

perform_analysis <- function(y_var, control_group_type, dataset, covariates = NULL) {
  data_filtered <- dataset %>%
    filter(first_order %notin% years_few_observations)

  if (!is.null(covariates) && length(covariates) > 0) {
    xformla <- as.formula(paste( "~", paste(covariates, collapse = " + ")))
  } else {
    xformla <- as.formula(paste( "~ 1"))

  }

  # Expected column names
  expected_columns <- c("bvd_id_numeric", "year", y_var, "first_order")

  # Check for missing columns
  missing_columns <- expected_columns[!expected_columns %in% colnames(data_filtered)]
  if (length(missing_columns) > 0) {
    stop(paste("The following columns are missing:", paste(missing_columns, collapse = ", ")))
  }

  cs_results <- att_gt(
    yname = y_var,
    tname = "year",
    idname = "bvd_id_numeric",
    gname = "first_order_2",
    xformla = xformla,
    data = data_filtered,
    control_group = control_group_type,
    est_method = "ipw",
    clustervars = "bvd_id_numeric",
    pl=TRUE,
    cores=8,
    bstrap = T,
    panel = TRUE,
    biters = 3000,
    allow_unbalanced_panel = TRUE,
  )

  # You can add further calculations if needed.


  # Compute the different results
  cs_results.dyn <- aggte(cs_results, type = "dynamic", na.rm = TRUE, max_e = 10, min_e = -10, cband = F)
  cs_results.sim <- aggte(cs_results, type = "simple", na.rm = TRUE)
  cs_results.grp <- aggte(cs_results, type = "group", na.rm = TRUE)
  cs_results.cal <- aggte(cs_results, type = "calendar", na.rm = TRUE)

  sim_ratio <- cs_results.sim$overall.att / cs_results.sim$overall.se

  # Visualization and confidence intervals for dynamic results
  cs_results_plot_dyn <- ggdid(cs_results.dyn)
  cs_results_data_dyn <- cs_results_plot_dyn$data
  cs_results_data_dyn$lower <- cs_results_data_dyn$att - cs_results_data_dyn$c * cs_results_data_dyn$att.se
  cs_results_data_dyn$upper <- cs_results_data_dyn$att + cs_results_data_dyn$c * cs_results_data_dyn$att.se

  # Create a list to store all the results
  results_list <- list(
    main = cs_results,
    dynamic = cs_results.dyn,
    simple = cs_results.sim,
    group = cs_results.grp,
    calendar = cs_results.cal,
    sim_ratio = sim_ratio,
    dynamic_plot_data = cs_results_data_dyn
  )

  return(results_list)
}


run_all_analyses_multiple_combinations <- function(y_vars, dataset) {

  results <- list()

  control_groups <- c('nevertreated', 'notyettreated')

  # Create all possible covariate combinations
  all_covariates <- c("pre_log_operating_revenue_turnover","pre_log_ebitda","pre_log_fixed_assets", "age")

  covariate_combinations <- list("none" = NULL)

  # Loop through all sizes of combinations
  for (i in 1:length(all_covariates)) {
    combinations <- combn(all_covariates, i)
    for (j in 1:ncol(combinations)) {
      combination_name <- paste(combinations[,j], collapse="_")
      covariate_combinations[[combination_name]] <- combinations[,j]
    }
  }

  # Calculate total number of iterations
  total_iterations <- length(y_vars) * length(control_groups) * length(covariate_combinations)

  # Create a progress bar
  pb <- progress_bar$new(total = total_iterations, format = "[:bar] :percent :elapsedfull")

  # Analyze data with each combination
  for (y_var in y_vars) {
    for (control_group in control_groups) {
      for (cov_name in names(covariate_combinations)) {
        covariates <- covariate_combinations[[cov_name]]
        results_key <- paste(y_var, control_group, cov_name, sep = "_")

        results[[results_key]] <- perform_analysis(y_var, control_group, dataset, covariates)

        # Update the progress bar
        pb$tick()
      }
    }
  }

  return(results)
}

In [10]:
full_panel$bvd_id_numeric <- as.numeric(as.factor(full_panel$bvd_id_number))
full_panel_ht <- full_panel %>% filter(first_order_tech==1)
full_panel_lt<- full_panel %>% filter(first_order_tech==0)
full_panel_one<- full_panel %>% filter(total_orders ==1)
full_panel_multiple<- full_panel %>% filter(total_orders >1)
full_panel_large_projects<- full_panel %>% filter(subproject_1_first_year %in% c("LHC", "HL"))
full_panel_less_than_100k<- full_panel %>% filter(first_order_amount <=100000)
full_panel_greater_than_100k<- full_panel %>% filter(first_order_amount >100000)
full_panel_SME<- full_panel %>% filter(SME_status ==1)

In [11]:
y_vars<- vars <- c( "probability_publications", "probability_applications",
                    "log_applications", "log_application_stock",
                    "log_weighted_patent_apps",
                   "log_publications", "log_publication_stock"

                   )

years_few_observations <- c(1995, 1996, 2008, 2020)

In [ ]:
cs_SME_results <- run_all_analyses_multiple_combinations(y_vars, full_panel_SME)


Streaming output truncated to the last 5000 lines.
Warning message in compute.att_gt(dp):
“No units in group 1998 in time period 1992”
Warning message in compute.att_gt(dp):
“No units in group 1998 in time period 1991”
Warning message in compute.att_gt(dp):
“No units in group 1998 in time period 1993”
Warning message in compute.att_gt(dp):
“No units in group 1998 in time period 1992”
Warning message in compute.att_gt(dp):
“No units in group 1998 in time period 1993”
Warning message in compute.att_gt(dp):
“No units in group 1999 in time period 1991”
Warning message in compute.att_gt(dp):
“No units in group 1999 in time period 1990”
Warning message in compute.att_gt(dp):
“No units in group 1999 in time period 1991”
Warning message in compute.att_gt(dp):
“No units in group 2000 in time period 1991”
Warning message in compute.att_gt(dp):
“No units in group 2000 in time period 1990”
Warning message in compute.att_gt(dp):
“No units in group 2000 in time period 1992”
Warning message in comput